In [ ]:
# !pip install -q tensorflow==2.16.1
# !pip install spacy==3.7.1
# !pip install -q datasets
# !pip install -q transformers
# !pip install -q jax[cpu]
# !pip install -q scipy==1.13.3
# !pip install -q pyarrow==14.0.2
# !pip install -q packaging==23.2
# !python -m spacy download en_core_web_sm
# !pip install keras==3.0.5
# !pip install keras-nlp==0.8.2

In [1]:
!pip cache purge

Files removed: 8


In [2]:
!pip install --upgrade pip
!pip install -q tensorflow==2.16.1
!pip install spacy==3.7.1
!pip install -q datasets
!pip install -q transformers
!pip install -q jax[cpu]
!pip install -q scipy==1.13.3
!pip install -q pyarrow==14.0.2
!pip install -q packaging==23.2
!python -m spacy download en_core_web_sm
!pip install keras==3.0.5
!pip install keras-nlp==0.8.2
#!pip install --upgrade gradio
!pip install gradio
!pip install Pillow
!pip install torchvision
!pip install SpeechRecognition
!pip install medmnist
!pip install librosa transformers
!pip install peft
!pip install accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 83.5 MB/s eta 0:00:00
  Attempting uninstall: typer
    Found existing installation: typer 0.12.3
    Uninstalling typer-0.12.3:
      Successfully uninstalled typer-0.12.3
  Attempting uninstall: smart-open
    Found existing installation: smart_open 7.0.4
    Uninstalling smart_open-7.0.4:
      Successfully uninstalled smart_open-7.0.4
  Attempting uninstall: cloudpathlib
    Found existing installation: cloudpathlib 0.19.0
    Uninstalling cloudpathlib-0.19.0:
      Successfully uninstalled cloudpathlib-0.19.0
  Attempting uninstall: weasel
    Found existing installation: weasel 0.4.1
    Uninstalling weasel-0.4.1:
      Successfully uninstalled weasel-0.4.1
  Attempting uninstall: spacy
    Found existing 

In [ ]:
#!pip install typer==0.12.5 weasel --force-reinstall


In [ ]:
# !pip install cupy-cuda11x ptxcompiler
# !pip install apache-beam==2.46.0 scipy==1.13.3
# !pip install numpy==1.25.0 pyarrow==16.1.0


In [3]:
import os
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import keras
import numpy as np
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from kaggle_secrets import UserSecretsClient
from itertools import islice
from datasets import load_dataset
import spacy
from transformers import AutoTokenizer, TFAutoModelForCausalLM
from sklearn.model_selection import train_test_split
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import speech_recognition as sr
from medmnist import PathMNIST, NoduleMNIST3D, ChestMNIST
import re
import librosa
import random
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, AutoTokenizer, AutoModelForCausalLM  
from peft import LoraConfig, get_peft_model
from transformers import Trainer, TrainingArguments

In [4]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("KAGGLE_KEY")
secret_value_1 = user_secrets.get_secret("KAGGLE_USERNAME")

# Hugging Face login 
from huggingface_hub import login
login(token="Huggin Face Token")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
# Constants for token length
MAX_INPUT_LENGTH = 512
MAX_OUTPUT_LENGTH = 128

# Function to examine text-based datasets
def examine_dataset(dataset_name, limit=10):
    dataset = load_dataset(dataset_name, split='train', streaming=True)
    sample = list(islice(dataset, limit))
    print(f"\nExamining {dataset_name}:")
    for i, example in enumerate(sample):
        print(f"\nExample {i + 1}:")
        print(example)
        print("\nKeys:", example.keys())
    return sample

# Updated Function to examine audio datasets with transcriptions
def examine_audio_dataset_with_labels(dataset, limit=10, dataset_type="common"):
    print(f"\nExamining {dataset.info.builder_name} dataset:")
    for i, example in enumerate(dataset):
        if i >= limit:
            break
        print(f"\nExample {i + 1}:")
        print(f"Audio shape: {example['audio']['array'].shape}")
        
        # For LibriSpeech, transcriptions are usually in 'text'
        if dataset_type == "librispeech" and 'text' in example:
            print(f"Text (Transcription): {example['text']}")
        
        # For Common Voice, transcriptions are usually in 'sentence'
        if dataset_type == "common" and 'sentence' in example:
            print(f"Sentence (Transcription): {example['sentence']}")
        else:
            print("No transcription found for this example.")
    print("\nKeys:", example.keys())

# Integrating LibriSpeech and Common Voice datasets for examination and training
def load_and_examine_audio_datasets():
    print("\nLoading and examining audio datasets:")
    
    # Load LibriSpeech dataset with correct configuration
    print("\nExamining LibriSpeech dataset:")
    librispeech = load_dataset("librispeech_asr", "clean", split="train.100", streaming=True)
    examine_audio_dataset_with_labels(librispeech, dataset_type="librispeech")
    
    # Load Common Voice 13.0 dataset (adjust the language code as needed)
    print("\nExamining Common Voice 13.0 dataset:")
   # Stream the dataset to avoid downloading all at once
    common_voice_lang = load_dataset("mozilla-foundation/common_voice_13_0", "en", split="train", streaming=True)

   # Convert the streaming dataset to a list or process it in chunks
    sample_data = list(common_voice_lang.take(100))  # Takes 100 samples

    return librispeech, common_voice_lang


# Main function to call everything
def main():
    # Examine text-based datasets for chatbot interactions and medical diagnosis
    print("Examining text-based datasets:")
    persona_sample = examine_dataset("AlekseyKorshuk/persona-chat")
    medqa_sample = examine_dataset("medalpaca/medical_meadow_medqa")
    meddia_sample = examine_dataset("wasiqnauman/medical-diagnosis-synthetic")

    # Load and examine audio datasets
    load_and_examine_audio_datasets()

    # After examining all datasets, proceed with standardization and model training
    print("\nDataset examination complete. Proceed with standardization and model training.")

if __name__ == "__main__":
    main()


Examining text-based datasets:


dataset_infos.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]


Examining AlekseyKorshuk/persona-chat:

Example 1:
{'personality': ['i like to remodel homes .', 'i like to go hunting .', 'i like to shoot a bow .', 'my favorite holiday is halloween .'], 'utterances': [{'candidates': ['my mom was single with 3 boys , so we never left the projects .', 'i try to wear all black every day . it makes me feel comfortable .', 'well nursing stresses you out so i wish luck with sister', 'yeah just want to pick up nba nfl getting old', 'i really like celine dion . what about you ?', 'no . i live near farms .', "i wish i had a daughter , i'm a boy mom . they're beautiful boys though still lucky", 'yeah when i get bored i play gone with the wind my favorite movie .', "hi how are you ? i'm eating dinner with my hubby and 2 kids .", 'were you married to your high school sweetheart ? i was .', 'that is great to hear ! are you a competitive rider ?', "hi , i'm doing ok . i'm a banker . how about you ?", "i'm 5 years old", 'hi there . how are you today ?', 'i totall

README.md:   0%|          | 0.00/1.77k [00:00<?, ?B/s]


Examining medalpaca/medical_meadow_medqa:

Example 1:
{'input': "Q:A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?? \n{'A': 'Ampicillin', 'B': 'Ceftriaxone', 'C': 'Ciprofloxacin', 'D': 'Doxycycline', 'E': 'Nitrofurantoin'},", 'instruction': 'Please answer with one of the option in the bracket', 'output': 'E: Nitrofurantoin'}

Keys: dict_keys(['input', 'instruction', 'output'])

Example 2:
{'input': "Q:A 3-month-old baby died suddenly at night while asleep. Hi

...

languages.py:   0%|          | 0.00/3.65k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

The repository for mozilla-foundation/common_voice_13_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_13_0.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Reading metadata...: 1013968it [00:23, 43026.58it/s]



Dataset examination complete. Proceed with standardization and model training.


In [ ]:
#Checking the Model's Layers
gemma_model = AutoModelForCausalLM.from_pretrained("google/gemma-2b")
print(gemma_model)  # This will print the model architecture


In [6]:
# Expanded symptom dictionary with follow-up questions, self-care tips, and warning signs
symptoms = {
    # Headache symptom block
    "headache": {
        "clarifying_questions": [
            "How long have you been experiencing the headache?",
            "Where is the pain located? (e.g., frontal, temporal, occipital, etc.)",
            "Is the pain throbbing, sharp, dull, or pressure-like?",
            "What is the intensity of the pain (on a scale of 1-10)?",
            "Have you noticed anything that makes it better or worse? (e.g., light, sound, movement, stress)",
            "Are there any other symptoms (e.g., nausea, vision changes, neck stiffness)?",
            "Do you have a history of similar headaches?",
            "Have you taken any medications or treatments for the headache?",
            "Do you have a history of migraines or head injuries?",
            "Do you feel comfortable discussing your pain level? Sometimes, pain perception can differ across individuals due to past healthcare experiences."

        ],
        "self_care": [
            "Rest in a quiet, dark room.",
            "Apply a cold or warm compress to your forehead.",
            "Stay hydrated.",
            "Take over-the-counter pain relievers like acetaminophen or ibuprofen (follow dosage instructions)."
        ],
        "severity_classification": "mild",  # Can be updated dynamically based on answers
        "warning_signs": [
            "Sudden, severe 'thunderclap' headache",
            "Headache with fever and stiff neck (possible meningitis)",
            "Headache after head trauma",
            "Headache with vision or speech problems",
            "The 'worst headache' you've ever experienced"
        ],
        "potential_diagnoses": ["Tension headache", "Migraine", "Cluster headache", "Sinusitis", "Concussion", "Meningitis"]
    },
    
    # Chest pain symptom block
    "chest pain": {
        "clarifying_questions": [
            "How long have you been experiencing chest pain?",
            "Where exactly is the pain located? (center, left, right of chest)",
            "Describe the pain: sharp, pressure-like, burning, crushing?",
            "What is the intensity of the pain (on a scale of 1-10)?",
            "Does the pain spread to other areas (e.g., arm, jaw, back)?",
            "Does the pain worsen with breathing, coughing, or physical activity?",
            "Are you experiencing shortness of breath, nausea, or dizziness?",
            "Do you have any other symptoms (e.g., palpitations, sweating, fatigue)?",
            "Do you have a history of heart disease or high blood pressure?",
            "Have you had experiences in the past where your pain was not taken seriously? It's important that we assess this fully."

        ],
        "self_care": [
            "Rest and avoid strenuous activity.",
            "Take slow, deep breaths.",
            "If prescribed, use nitroglycerin as directed."
        ],
        "severity_classification": "severe",  # Default, should update dynamically
        "warning_signs": [
            "Severe, crushing chest pain",
            "Pain radiating to the arm, jaw, or back",
            "Shortness of breath",
            "Sweating, nausea, or vomiting"
        ],
        "potential_diagnoses": ["Heart attack", "Angina", "Pleurisy", "Pericarditis", "Gastroesophageal reflux disease (GERD)", "Musculoskeletal pain"]
    },

    # Abdominal pain symptom block
    "abdominal pain": {
        "clarifying_questions": [
            "Where in your abdomen is the pain located?",
            "Is the pain sharp, dull, cramping, or bloating?",
            "How would you rate the intensity on a scale of 1-10?",
            "Is the pain constant or does it come and go?",
            "Have you noticed any other symptoms (e.g., nausea, vomiting, diarrhea, constipation)?",
            "Have you noticed any blood in your stool or urine?",
            "Do you have any history of gastrointestinal problems or surgeries?",
            "Does eating or avoiding certain foods affect the pain?"
        ],
        "self_care": [
            "Rest and avoid solid foods until symptoms improve.",
            "Stay hydrated with clear fluids (water, broth).",
            "Use a heating pad on a low setting for cramping.",
            "Take over-the-counter antacids for indigestion or mild discomfort."
        ],
        "severity_classification": "moderate",
        "warning_signs": [
            "Severe, persistent pain",
            "Inability to keep food or fluids down",
            "Signs of dehydration",
            "Abdominal swelling or tenderness",
            "Fever above 101°F (38.3°C)"
        ],
    },

    # Shortness of breath symptom block
    "shortness of breath": {
        "clarifying_questions": [
            "How long have you been experiencing shortness of breath?",
            "Does it happen during rest, physical activity, or both?",
            "Do you feel any tightness or pain in your chest?",
            "Are there any other symptoms (e.g., wheezing, cough, dizziness)?",
            "Do you have a history of asthma, COPD, or heart problems?",
            "Have you been exposed to allergens or irritants recently?"
        ],
        "self_care": [
            "Sit upright and lean forward slightly.",
            "Try pursed-lip breathing.",
            "Use a fan to circulate air.",
            "Avoid smoking or exposure to secondhand smoke."
        ],
        "severity_classification": "moderate to severe",
        "warning_signs": [
            "Sudden onset of severe breathlessness",
            "Blue tinge to lips or fingernails",
            "Inability to speak in full sentences",
            "Chest pain or pressure",
            "Dizziness or fainting"
        ],
    },

    # Cough symptom block
    "cough": {
        "clarifying_questions": [
            "How long have you had the cough?",
            "Is the cough dry or productive (bringing up mucus)?",
            "What color is the mucus? (clear, yellow, green, bloody)",
            "Are there any other symptoms (e.g., fever, shortness of breath, chest pain)?",
            "Have you been in contact with anyone who has a respiratory illness?"
        ],
        "self_care": [
            "Drink plenty of fluids.",
            "Get rest.",
            "Use over-the-counter cough medications (suppressants or expectorants)."
        ],
        "severity_classification": "mild",
        "warning_signs": [
            "Coughing up blood",
            "Severe shortness of breath",
            "Persistent high fever",
            "Cough lasting more than 3 weeks"
        ],
    },

    # Fever symptom block
    "fever": {
        "clarifying_questions": [
            "What is your current temperature?",
            "How long have you had the fever?",
            "Are you experiencing any other symptoms (e.g., chills, body aches)?",
            "Have you recently traveled or been exposed to any illnesses?",
            "Are you taking any medications that might affect your temperature?"
        ],
        "self_care": [
            "Rest and stay hydrated.",
            "Take acetaminophen or ibuprofen to reduce the fever.",
            "Use a lukewarm compress.",
            "Dress in light, breathable clothing."
        ],
        "severity_classification": "mild to moderate",
        "warning_signs": [
            "Fever above 103°F (39.4°C)",
            "Fever lasting more than 3 days",
            "Severe headache or neck stiffness",
            "Confusion or decreased alertness",
            "Difficulty breathing or chest pain"
        ],
    # Mental Health
        "anxiety": {
    "clarifying_questions": [
        "How long have you been feeling anxious?",
        "Do you often feel anxious in specific situations (e.g., social gatherings, work)?",
        "Do you experience physical symptoms with your anxiety (e.g., racing heart, sweating, shortness of breath)?",
        "Have you noticed any changes in your sleeping or eating habits?",
        "Have you experienced any panic attacks or moments of extreme fear recently?",
        "Are you currently taking any medication or receiving therapy for anxiety?",
        "Are there specific triggers that cause your anxiety to worsen?"
    ],
    "self_care": [
        "Practice deep breathing exercises or mindfulness.",
        "Engage in regular physical activity.",
        "Avoid caffeine and alcohol, which can exacerbate anxiety symptoms.",
        "Consider talking to a friend or family member for support.",
        "Maintain a regular sleep schedule."
    ],
    "severity_classification": "mild to moderate",  # Dynamic based on input
    "warning_signs": [
        "Persistent or worsening anxiety despite self-care efforts",
        "Frequent panic attacks",
        "Inability to perform daily tasks due to anxiety",
        "Thoughts of self-harm or suicide"
    ],
}

    }
    # Add more symptoms as needed for a comprehensive database.
}


urgency_levels = ["immediate", "urgent", "routine", "self_care"]
context = {"symptom": None, "severity": None, "urgency": None, "duration": None, "answers": []}

# Correct split names for streaming datasets
librispeech = load_dataset("librispeech_asr", "clean", split="train.100", streaming=True)
common_voice_lang = load_dataset("mozilla-foundation/common_voice_13_0", "en", split="train", streaming=True)

# Load Wav2Vec2 model for speech-to-text
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
wav2vec_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# Load the Gemma tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
gemma_model = AutoModelForCausalLM.from_pretrained("google/gemma-2b")

# Fine-tuning the Gemma model with LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["self_attn.q_proj", "self_attn.k_proj", "self_attn.v_proj", "self_attn.o_proj"],
    lora_dropout=0.05
)
gemma_model = get_peft_model(gemma_model, lora_config)

# Fine-tuning Wav2Vec2 model
def prepare_dataset(batch):
    audio = batch["audio"]["array"]
    batch["input_values"] = processor(audio, sampling_rate=16000).input_values[0]
    batch["labels"] = processor.tokenizer(batch["text"]).input_ids
    return batch

def fine_tune_wav2vec2(dataset):
    dataset = dataset.map(prepare_dataset, remove_columns=["audio", "text"])
    dataset = dataset.train_test_split(test_size=0.1)
    
    training_args = TrainingArguments(
        output_dir="./wav2vec2-finetuned",
        per_device_train_batch_size=16,
        evaluation_strategy="steps",
        num_train_epochs=3,
        save_steps=100,
        eval_steps=100,
        logging_steps=100,
        learning_rate=5e-5,
        save_total_limit=1,
    )
    
    trainer = Trainer(
        model=wav2vec_model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        tokenizer=processor.feature_extractor,
    )
    
    trainer.train()
    return wav2vec_model

# Text dataset standardization
def prepare_text_dataset(dataset_name, limit=10):
    dataset = load_dataset(dataset_name, split='train', streaming=True)
    samples = list(islice(dataset, limit))
    
    print(f"\nStandardizing {dataset_name}:")
    standardized_data = []
    
    for sample in samples:
        text = sample.get('text', '') or sample.get('sentence', '')
        standardized_text = clean_text(text)
        truncated_text = truncate_text(standardized_text, 512)
        standardized_data.append(truncated_text)
    
    return standardized_data

# Audio dataset standardization
def prepare_audio_dataset(dataset_name, name="clean", split_name="train.100", limit=10):
    dataset = load_dataset(dataset_name, name=name, split=split_name, streaming=True)
    samples = list(islice(dataset, limit))
    
    print(f"\nStandardizing {dataset_name} (audio):")
    standardized_data = []
    
    for sample in samples:
        audio = sample['audio']['array']
        transcription = sample.get('text', '') or sample.get('sentence', '')
        standardized_text = clean_text(transcription)
        truncated_text = truncate_text(standardized_text, 128)
        processed_audio = processor(audio, sampling_rate=16000).input_values[0]
        standardized_data.append((processed_audio, truncated_text))
    
    return standardized_data

# Standardization and dataset preparation
def integrate_standardization():
    print("Starting dataset standardization process...")
    
    # Standardize text-based datasets
    persona_chat = prepare_text_dataset("AlekseyKorshuk/persona-chat")
    medqa_dataset = prepare_text_dataset("medalpaca/medical_meadow_medqa")
    meddia_sample = prepare_text_dataset("wasiqnauman/medical-diagnosis-synthetic")
    
    # Standardize audio-based datasets
    librispeech_data = prepare_audio_dataset("librispeech_asr", name="clean", split_name="train.100")
    common_voice_data = prepare_audio_dataset("mozilla-foundation/common_voice_13_0", name="en", split_name="train", limit=20)
    
    print("\nStandardization complete.")
    
# Text cleaning and processing
def clean_text(text: str) -> str:
    return re.sub(r'\s+', ' ', text).strip()

def truncate_text(text: str, max_length: int) -> str:
    if len(text) <= max_length:
        return text
    return text[:text.rfind(' ', 0, max_length)]

# Classify Symptom Severity Function 
def classify_symptom_severity(symptom, answers):
    """
    Classifies the severity of a symptom based on predefined criteria, such as pain intensity,
    duration, warning signs, and accompanying symptoms.
    """
    severity = "mild"  # Default to mild

    # Get the warning signs and severity classifications for the given symptom
    warning_signs = symptoms[symptom].get('warning_signs', [])
    severity_classification = symptoms[symptom].get('severity_classification', "mild")

    # Step 1: Check if any warning signs are mentioned in the patient's answers
    for warning_sign in warning_signs:
        if warning_sign.lower() in answers.lower():
            return "severe"  # If any warning sign is present, mark as severe

    # Step 2: Assess based on predefined severity classification (default from symptom dictionary)
    if severity_classification == "severe":
        severity = "severe"
    elif severity_classification == "moderate":
        severity = "moderate"
    elif severity_classification == "mild":
        severity = "mild"

    # Step 3: Evaluate pain intensity or duration (if provided in answers)
    if "pain" in answers.lower():
        pain_scale = [int(s) for s in answers.split() if s.isdigit() and int(s) <= 10]
        if pain_scale and max(pain_scale) >= 7:
            severity = "severe"  # Pain scale >= 7 is classified as severe
        elif pain_scale and 4 <= max(pain_scale) < 7:
            severity = "moderate"
        else:
            severity = "mild"

    # Step 4: Consider the duration of symptoms (if provided)
    if "week" in answers or "weeks" in answers or "several" in answers:
        severity = "moderate to severe"
    elif "day" in answers or "days" in answers:
        severity = "moderate"
    elif "hours" in answers:
        severity = "mild"

    return severity


# Generate triage conversation flow
def generate_complex_triage_conversation(symptom):
    greeting = "Hi, how can I assist you today?"
    patient_message = f"I've been experiencing {symptom}. Should I be concerned?"

    bot_response = express_empathy(symptom)
    clarifying_questions = gather_symptom_details(symptom)  # This pulls clarifying questions from the symptoms dictionary
    patient_answers = generate_patient_answers(symptom, clarifying_questions)  # Pass both symptom and clarifying questions
    
    pain_scale_question = "On a scale of 1-10, what is your pain like?"
    pain_scale_answer = random.choice(["It's about a 7.", "It's a 3, not too bad but persistent."])
    
    severity = classify_symptom_severity(symptom, patient_answers) # Determines severity
    duration = random.choice(["a few hours", "a couple of days", "about a week", "several weeks"])
    update_context(symptom, severity, None, duration, patient_answers)  # Updates the context with severity, answers, etc.
    final_response = make_recommendation(symptom, severity, duration, patient_answers, pain_scale_answer)  # Final bot recommendation

    # Construct the conversation flow
    conversation = f"""Pearly: {greeting}
User: {patient_message}
Pearly: {bot_response}
Pearly: {clarifying_questions}
User: {patient_answers}
Pearly: {pain_scale_question}
User: {pain_scale_answer}
Pearly: {final_response}"""

    return conversation

# Empathy and symptom details gathering
def express_empathy(symptom):
    return f"I'm so sorry to hear you're experiencing {symptom}. Let's gather more information to help you."

def gather_symptom_details(symptom):
    clarifying_questions = symptoms[symptom]['clarifying_questions']
    return " ".join(clarifying_questions)

# Context update and recommendation
def update_context(symptom, severity, urgency, duration, answers):
    context['symptom'] = symptom
    context['severity'] = severity
    context['urgency'] = urgency
    context['duration'] = duration
    context['answers'] = answers

def make_recommendation(symptom, severity, duration, answers, pain_scale):
    urgency = determine_urgency(symptom, severity, duration)
    update_context(symptom, severity, urgency, duration, answers)
    
    if urgency == "immediate":
        return f"Based on your pain level of {pain_scale}, I recommend seeking immediate medical attention for your {symptom}."
    elif urgency == "urgent":
        return f"It sounds important to get your {symptom} checked soon. I can help you schedule an appointment with a GP."
    elif urgency == "routine":
        return f"Your {symptom} doesn't seem urgent, but it's a good idea to see a doctor if you're concerned."
    else:
        return f"It seems like your {symptom} can be managed at home. Here's what you can try: {', '.join(symptoms[symptom]['self_care'])}. If symptoms worsen, please reach out."

def determine_urgency(symptom, severity, duration):
    if severity == "severe" or symptom in ["chest pain", "difficulty breathing"]:
        return "immediate"
    elif severity == "moderate" or duration in ["about a week", "several weeks"]:
        return "urgent"
    elif severity == "mild" and duration in ["a few hours", "a couple of days"]:
        return "self_care"
    else:
        return "routine"

# Main function integrating the logic
def main():
    integrate_standardization()  # Run the dataset standardization
    
    # Example audio input
    dummy_audio_signal = np.random.rand(16000 * 2)  # 2-second dummy audio
    chatbot_response = handle_user_audio(dummy_audio_signal)
    print(chatbot_response)
    
symptoms = {
    # Headache symptom block
    "headache": {
        "clarifying_questions": [
            "How long have you been experiencing the headache?",
            "Where is the pain located? (e.g., frontal, temporal, occipital, etc.)",
            "Is the pain throbbing, sharp, dull, or pressure-like?",
            "What is the intensity of the pain (on a scale of 1-10)?",
            "Have you noticed anything that makes it better or worse? (e.g., light, sound, movement, stress)?",
            "Are there any other symptoms (e.g., nausea, vision changes, neck stiffness)?",
            "Do you have a history of similar headaches?",
            "Have you taken any medications or treatments for the headache?",
            "Do you have a history of migraines or head injuries?",
            "Do you feel comfortable discussing your pain level?"
        ],
        "self_care": [
            "Rest in a quiet, dark room.",
            "Apply a cold or warm compress to your forehead.",
            "Stay hydrated.",
            "Take over-the-counter pain relievers like acetaminophen or ibuprofen (follow dosage instructions)."
        ],
        "severity_classification": "mild",
        "warning_signs": [
            "Sudden, severe 'thunderclap' headache",
            "Headache with fever and stiff neck (possible meningitis)",
            "Headache after head trauma",
            "Headache with vision or speech problems",
            "The 'worst headache' you've ever experienced"
        ],
        "potential_diagnoses": ["Tension headache", "Migraine", "Cluster headache", "Sinusitis", "Concussion", "Meningitis"]
    },
    
    # Chest pain symptom block
    "chest pain": {
        "clarifying_questions": [
            "How long have you been experiencing chest pain?",
            "Where exactly is the pain located? (center, left, right of chest)?",
            "Describe the pain: sharp, pressure-like, burning, crushing?",
            "What is the intensity of the pain (on a scale of 1-10)?",
            "Does the pain spread to other areas (e.g., arm, jaw, back)?",
            "Does the pain worsen with breathing, coughing, or physical activity?",
            "Are you experiencing shortness of breath, nausea, or dizziness?",
            "Do you have any other symptoms (e.g., palpitations, sweating, fatigue)?",
            "Do you have a history of heart disease or high blood pressure?",
            "Have you had experiences where your pain wasn't taken seriously?"
        ],
        "self_care": [
            "Rest and avoid strenuous activity.",
            "Take slow, deep breaths.",
            "If prescribed, use nitroglycerin as directed."
        ],
        "severity_classification": "severe",
        "warning_signs": [
            "Severe, crushing chest pain",
            "Pain radiating to the arm, jaw, or back",
            "Shortness of breath",
            "Sweating, nausea, or vomiting"
        ],
        "potential_diagnoses": ["Heart attack", "Angina", "Pleurisy", "Pericarditis", "GERD", "Musculoskeletal pain"]
    },

    # Abdominal pain symptom block
    "abdominal pain": {
        "clarifying_questions": [
            "Where in your abdomen is the pain located?",
            "Is the pain sharp, dull, cramping, or bloating?",
            "How would you rate the intensity on a scale of 1-10?",
            "Is the pain constant or does it come and go?",
            "Have you noticed any other symptoms (e.g., nausea, vomiting, diarrhea, constipation)?",
            "Have you noticed any blood in your stool or urine?",
            "Do you have any history of gastrointestinal problems or surgeries?",
            "Does eating or avoiding certain foods affect the pain?"
        ],
        "self_care": [
            "Rest and avoid solid foods until symptoms improve.",
            "Stay hydrated with clear fluids (water, broth).",
            "Use a heating pad on a low setting for cramping.",
            "Take over-the-counter antacids for indigestion or mild discomfort."
        ],
        "severity_classification": "moderate",
        "warning_signs": [
            "Severe, persistent pain",
            "Inability to keep food or fluids down",
            "Signs of dehydration",
            "Abdominal swelling or tenderness",
            "Fever above 101°F (38.3°C)"
        ],
    },

    # Shortness of breath symptom block
    "shortness of breath": {
        "clarifying_questions": [
            "How long have you been experiencing shortness of breath?",
            "Does it happen during rest, physical activity, or both?",
            "Do you feel any tightness or pain in your chest?",
            "Are there any other symptoms (e.g., wheezing, cough, dizziness)?",
            "Do you have a history of asthma, COPD, or heart problems?",
            "Have you been exposed to allergens or irritants recently?"
        ],
        "self_care": [
            "Sit upright and lean forward slightly.",
            "Try pursed-lip breathing.",
            "Use a fan to circulate air.",
            "Avoid smoking or exposure to secondhand smoke."
        ],
        "severity_classification": "moderate to severe",
        "warning_signs": [
            "Sudden onset of severe breathlessness",
            "Blue tinge to lips or fingernails",
            "Inability to speak in full sentences",
            "Chest pain or pressure",
            "Dizziness or fainting"
        ],
    }
}


# Gather symptom details by accessing the clarifying questions
def gather_symptom_details(symptom):
    clarifying_questions = symptoms[symptom]['clarifying_questions']
    return " ".join(clarifying_questions)

# Generate patient answers based on the symptom and the provided questions
def generate_patient_answers(symptom, questions):
    # Default to some random answers for each symptom
    symptom_specific_answers = {
        "headache": [
            "It started about two days ago.",
            "The pain is mostly in my temples and forehead.",
            "It's a throbbing pain, about 6 out of 10 in intensity."
            # Add more responses based on the original symptoms
        ],
        # Add other symptoms like "chest pain", "abdominal pain", etc.
    }

    if symptom in symptom_specific_answers:
        possible_answers = symptom_specific_answers[symptom]
    else:
        possible_answers = [
            "I'm not sure, it's hard to describe.",
            "It's been ongoing for a while.",
            "The symptoms come and go.",
            "This is the first time I've experienced this."
        ]

    num_answers = random.randint(1, min(len(questions), len(possible_answers)))
    answers = random.sample(possible_answers, num_answers)

    return " ".join(answers)


# Example usage:
questions = ["How long have you had the symptom?", "What are the related symptoms?"]
print("Patient Response for Headache:")
print(generate_patient_answers("headache", questions))

print("\nPatient Response for Chest Pain:")
print(generate_patient_answers("chest pain", questions))

  
    
    # Tokenization examples after conversation generation
conversation_data = [generate_complex_triage_conversation(random.choice(list(symptoms.keys()))) for _ in range(3)]
tokenization_ready_data = [item for item in conversation_data]
    
tokenized_data = tokenizer(tokenization_ready_data, padding=True, truncation=True, max_length=512, return_tensors="pt")

    # Print some tokenized examples
for i in range(3):
    print(f"\nExample {i+1}:")
    print(tokenizer.decode(tokenized_data.input_ids[i]))
    print("Input shape:", tokenized_data.input_ids[i].shape)

    print("\nShape of input_ids:", tokenized_data.input_ids.shape)

# Placeholder function for handling user audio (to be replaced with actual speech-to-text implementation)
def handle_user_audio(audio_signal):
    try:
        inputs = processor(audio_signal, sampling_rate=16000, return_tensors="pt").input_values
        with torch.no_grad():
            logits = wav2vec_model(inputs).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.batch_decode(predicted_ids)[0]
        return transcription
    except Exception as e:
        return f"Error processing audio: {e}"

disclaimer = "Please note that this chatbot provides general health information and does not replace the advice of a healthcare professional. For serious or concerning symptoms, consult with a doctor."

if __name__ == "__main__":
    main()



preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Patient Response for Headache:
The pain is mostly in my temples and forehead. It started about two days ago.

Patient Response for Chest Pain:
I'm not sure, it's hard to describe. It's been ongoing for a while.

Example 1:
<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><bos>Pearly: Hi, how can I assist you today?
User: I've been experiencing abdominal pain. Should I be concerned?
Pearly: I'm so sorry to hear you're experiencing abdominal pain. Let's gather more information to help you.
Pearly: Where in your abdomen is the pain located? Is the pain sharp, dull, cramping, or bloating? How would you rate the intensity on a scale of 1-10? Is the pain constant or does it come and go? Have you noticed any other symptoms (e.g., nausea, vomiting, diarrhea, constipation)? Have you noticed any blood in your stool or urin

Reading metadata...: 1013968it [00:18, 56168.39it/s]



Standardizing mozilla-foundation/common_voice_13_0 (audio):

Standardization complete.



In [8]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

!git --version


git version 2.34.1


In [9]:
# Configure Git 
!git config --global user.name "PearlIsa"
!git config --global user.email "pearlochisa@gmail.com"


In [10]:
# Initialize Git in the Directory
!cd /kaggle/working && git init


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /kaggle/working/.git/


In [11]:
# Set up GitHub repository as the remote:
!git remote add origin https://github.com/PearlIsa/Medical_triaging_chatbot_Kagglex_2024_Cohort4.git


In [24]:
# Check the status of the files
# !git status

# # Rename branch from master to main:
# !git branch -M main

# # Stage all the files you want to commit
# !git add .

# # Commit changes with a descriptive message
# !git commit -m "Initial commit for Medical triaging chatbot"

# # Push the changes to GitHub repository
# !git push origin main


On branch main

No commits yet

nothing to commit (create/copy files and use "git add" to track)
On branch main

Initial commit

nothing to commit (create/copy files and use "git add" to track)
error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/PearlIsa/Medical_triaging_chatbot_Kagglex_2024_Cohort4.git'


In [ ]:
!pip install --upgrade gradio

In [ ]:
import gradio as gr

# chatbot function which processes symptom text input
def chatbot(symptom_text):
    # Use your existing chatbot logic for text-based symptom processing
    conversation = generate_complex_triage_conversation(symptom_text)  
    return conversation

# function to handle audio input using Wav2Vec2
def handle_audio(audio):
    # Process the audio input through Wav2Vec2 for transcription
    try:
        audio_data = audio  # Gradio now returns the audio data directly
        inputs = processor(audio_data, sampling_rate=16000, return_tensors="pt").input_values
        with torch.no_grad():
            logits = wav2vec_model(inputs).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.batch_decode(predicted_ids)[0]
        return transcription
    except Exception as e:
        return f"Error processing audio: {e}"

# Combined function to handle both audio and text inputs
def chatbot_with_audio(symptom_text, audio):
    if audio is not None:
        # If audio is provided, process it using Wav2Vec2 model
        transcription = handle_audio(audio)
        response = f"Transcribed Audio: {transcription}\n\n" + chatbot(symptom_text)
    else:
        # Only handle the text input if no audio is provided
        response = chatbot(symptom_text)
    return response

# Set up Gradio interface with both audio and text inputs
interface = gr.Interface(
    fn=chatbot_with_audio,
    inputs=[
        gr.Audio(type="numpy", label="Audio Input (optional)"),  # For audio input
        gr.Textbox(lines=2, placeholder="Describe your symptom...", label="Symptom Description")
    ],
    outputs="text",  # Text output to display the conversation and transcription
    title="Medical Triage Chatbot with Audio",
    description="Enter a symptom and optionally provide an audio description. The chatbot will assist with possible next steps."
)

# Launch the demo
interface.launch (share=True)
